<a href="https://colab.research.google.com/github/carlavilla/DataViz2025/blob/main/CV_PS1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Merging data on homelessness count

In [1]:
from google.colab import drive
drive.mount('/content/drive')
with open('/content/drive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/drive/My\ Drive/foo.txt
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')
#this was my attempt at accessing the data on my GDrive. The alternative link provided in the notes solved it instead.
# What was this mount/unmount code supposed to do?

MessageError: Error: credential propagation was unsuccessful

aok: this is asking to acces MY goog drive so doesnt work, just load from online like we did in class and may see
https://theaok.github.io/generic/howToPutDataOnline.html

##Environment set up

In [2]:
import time, os, sys, re #basics
import zipfile, json, datetime, string
import numpy as np
from statistics import *
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler

In [3]:
import urllib

from google.colab import files
import seaborn as sns
from google.colab import data_table
data_table.enable_dataframe_formatter()

##Retrieving Data Sets

###First data set: PIT Count
Overall count of homeless individuals at PIT counts in Philadelphia, available 2007 to 2023 <br>
Retrieved from: https://www.hudexchange.info/resource/3031/pit-and-hic-data-since-2007/ <br>
Format: xlsb dataset that allows creation of a subtable. Subtable was extracted for PA-500 CoC Philadelphia as an xlsx file.

In [4]:
urllib.request.urlretrieve("https://drive.google.com/uc?id=1Rt9o407BcPjRLF2qzJljDHXqjLjWH0kj&export=download", "HUDPIT_PA500_2007to2023.xlsx")

('HUDPIT_PA500_2007to2023.xlsx', <http.client.HTTPMessage at 0x795582d46c50>)

In [5]:
pit = pd.read_excel('HUDPIT_PA500_2007to2023.xlsx', sheet_name= 'PA500_total')

In [ ]:
pit.columns

Index(['year', 'total_count'], dtype='object')

In [ ]:
pit.head(2)

,year,total_count
0,2007,3869
1,2008,3479


In [6]:
pit = pit.rename(columns={'year': 'Year'})

###Second data set: Temperature
Mean for minimum temperature per month 2007-2023 <br>
Retrieved from: https://www.weather.gov/wrh/Climate?wfo=phi <br>
Format: pdf

####First attempt: reading off pdf

In [7]:
%pip install tabula-py
import tabula

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 65.3 MB/s eta 0:00:00


In [10]:
urllib.request.urlretrieve("https://drive.google.com/uc?id=14E0MRpMx77aHd1MEVPOb-r3eug-Mr4hM&export=download", "min_temp_2007to2023.pdf")
#aok: i had to uncomment this to run

('min_temp_2007to2023.pdf', <http.client.HTTPMessage at 0x7955822cb190>)

In [11]:
from tabula import read_pdf
temp = read_pdf("min_temp_2007to2023.pdf", pages='all', lattice=True)

In [12]:
tabula.convert_into("min_temp_2007to2023.pdf", "min_temp_2007to2023.csv", output_format="csv", pages='all')

In [13]:
tempcsv = pd.read_csv('min_temp_2007to2023.csv')

EmptyDataError: No columns to parse from file

In [ ]:
tempcsv[0]

TypeError: 'NoneType' object is not subscriptable

####Second attempt: using adobe to convert pdf to xlsx

In [14]:
urllib.request.urlretrieve("https://drive.google.com/uc?id=1myHP4UxOXomqz5JSAYJyo5L3R5v-KW_b&export=download", "min_temp_2007to2023.xlsx")
temp=pd.read_excel('min_temp_2007to2023.xlsx')

In [15]:
temp.head(3)

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual
0,2007,30.7,21.1,33.4,41.9,55.2,64.8,68.6,68.3,62.3,55.7,38.3,31.8,47.7
1,2008,28.6,28.3,35.7,46.8,51.4,66.7,70.4,65.6,62.2,47.6,39.1,30.8,47.8
2,2009,22.6,28.1,35.0,45.6,55.9,63.2,66.8,70.3,60.3,48.6,44.0,29.4,47.5


###Third Dataset: Demolitions

Number of property demolitions initiated in Philadelphia 2007 to 2025 <br>
Retrieved from: https://opendataphilly.org/datasets/building-demolitions/ <br>
Format: csv

In [16]:
import pandas as pd
urllib.request.urlretrieve("https://drive.google.com/uc?id=1i_Uc-gNSdaHFVDcaCbX4o2CN04EvNEx7&export=download", "demolitions_2007to2025.xlsx")
demo=pd.read_excel('demolitions_2007to2025.xlsx', sheet_name='Sheet1')

In [17]:
demo.columns[2]

'Demolition count'

In [ ]:
demo.head(3)

,year,year pit,Demolition count
0,2007,2008,315
1,2008,2009,667
2,2009,2010,548


In [18]:
demo.drop(columns=['year'], inplace=True)

In [ ]:
demo.columns

Index(['year pit', 'Demolition count'], dtype='object')

In [19]:
demo.rename(columns={'year pit': 'Year'}, inplace=True)
demo.columns

Index(['Year', 'Demolition count'], dtype='object')

##Merging Datasets


###Merging Pit and Temp



In [20]:
pit_temp = pd.merge(pit, temp, how='outer', on=['Year'],indicator=True) #outer=keep all
pit_temp

,Year,total_count,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual,_merge
0,2007,3869,30.7,21.1,33.4,41.9,55.2,64.8,68.6,68.3,62.3,55.7,38.3,31.8,47.7,both
1,2008,3479,28.6,28.3,35.7,46.8,51.4,66.7,70.4,65.6,62.2,47.6,39.1,30.8,47.8,both
2,2009,3054,22.6,28.1,35.0,45.6,55.9,63.2,66.8,70.3,60.3,48.6,44.0,29.4,47.5,both
3,2010,3077,26.2,25.7,39.1,47.6,58.0,68.8,72.6,70.4,63.7,50.3,40.2,26.4,49.1,both
4,2011,3328,23.1,28.3,35.5,47.2,57.8,66.1,72.8,67.8,63.7,49.2,42.4,35.2,49.1,both
5,2012,2976,29.9,33.1,42.7,45.4,59.2,63.7,72.7,70.4,62.3,52.0,36.5,36.5,50.4,both
6,2013,3070,29.4,28.5,33.4,44.9,55.4,65.6,73.1,67.3,58.5,52.2,36.9,31.7,48.1,both
7,2014,3109,19.4,24.9,29.6,43.5,56.0,65.3,69.2,66.1,62.2,52.2,37.2,35.1,46.7,both
8,2015,3346,23.9,17.5,30.5,45.4,59.3,66.5,71.3,70.0,65.4,50.0,44.7,44.2,49.1,both
9,2016,3430,26.1,30.8,41.2,44.5,54.8,64.7,72.5,72.4,64.8,51.7,41.1,32.1,49.7,both


In [21]:
tempJan= temp[['Year','Jan']]
pit_temp2 = pd.merge(pit, tempJan, how='outer', on=['Year'],indicator=True) #outer=keep all
pit_temp2

,Year,total_count,Jan,_merge
0,2007,3869,30.7,both
1,2008,3479,28.6,both
2,2009,3054,22.6,both
3,2010,3077,26.2,both
4,2011,3328,23.1,both
5,2012,2976,29.9,both
6,2013,3070,29.4,both
7,2014,3109,19.4,both
8,2015,3346,23.9,both
9,2016,3430,26.1,both


###Merging Pit, Temp, and Demo

In [ ]:
pit3 = pd.merge(pit_temp2, demo, how='outer', on=['Year'],indicator='_merge2') #outer=keep all
pit3

,Year,total_count,Jan,_merge,Demolition count,_merge2
0,2007,3869.0,30.7,both,NaN,left_only
1,2008,3479.0,28.6,both,315.0,both
2,2009,3054.0,22.6,both,667.0,both
3,2010,3077.0,26.2,both,548.0,both
4,2011,3328.0,23.1,both,758.0,both
5,2012,2976.0,29.9,both,803.0,both
6,2013,3070.0,29.4,both,974.0,both
7,2014,3109.0,19.4,both,703.0,both
8,2015,3346.0,23.9,both,935.0,both
9,2016,3430.0,26.1,both,762.0,both


##Funsies Merge

###Fourt Dataset: Precipitation

In [ ]:
urllib.request.urlretrieve("https://drive.google.com/uc?id=1vm5e50vTZJoyGvA75OtmboqdpHtNWtHA&export=download", "mean_precip_2007to2023.xlsx")
precip = pd.read_excel('mean_precip_2007to2023.xlsx')
precip.head(3)

,"2/13/25, 12:49 AM about:blank",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,Monthly Mean Precipitation for Philadelphia A...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual
2,NaN,2007,0.11,0.06,0.12,0.3,0.09,0.13,0.11,0.09,0.02,0.15,0.05,0.14,0.11


In [ ]:
precip.tail(7)

,"2/13/25, 12:49 AM about:blank",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
18,NaN,2023,0.11,0.05,0.06,0.17,0.01,0.14,0.17,0.1,0.2,0.02,0.09,0.26,0.12
19,NaN,2024,0.19,0.05,0.23,0.14,0.06,0.16,0.08,0.16,0.03,T,0.08,0.11,0.11
20,NaN,Mean,0.1,0.11,0.12,0.13,0.11,0.15,0.15,0.17,0.14,0.11,0.09,0.15,0.13
21,NaN,Max,0.19,0.22,0.24,0.3,0.21,0.35,0.43,0.62,0.34,0.19,0.3,0.29,0.18
22,NaN,NaN,2024,2018,2010,2007,2016,2013,2013,2011,2011,2022,2018,2009,NaN
23,NaN,Min,0.05,0.03,0.03,0.06,0.01,0.06,0.05,0.03,0.02,T,0.02,0.04,0.1
24,NaN,NaN,2021,2009,2012,2016,2023,2017,2012,2015,2007,2024,2021,2017,NaN


In [ ]:
precip.drop(index=[1])
precip.head(5)

,"2/13/25, 12:49 AM about:blank",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
1,NaN,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual
2,NaN,2007,0.11,0.06,0.12,0.3,0.09,0.13,0.11,0.09,0.02,0.15,0.05,0.14,0.11
3,NaN,2008,0.06,0.14,0.12,0.07,0.15,0.1,0.11,0.08,0.14,0.05,0.13,0.18,0.11
4,NaN,2009,0.09,0.03,0.05,0.13,0.16,0.16,0.11,0.33,0.12,0.18,0.07,0.29,0.14
5,NaN,2010,0.07,0.21,0.24,0.09,0.08,0.07,0.2,0.07,0.12,0.16,0.06,0.1,0.12


In [ ]:
precip.rename(columns={'Unnamed: 1': 'Year', 'Unnamed: 2': 'Jan'})
precip.drop(index=[1])
precip.head(5)


,"2/13/25, 12:49 AM about:blank",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
1,NaN,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual
2,NaN,2007,0.11,0.06,0.12,0.3,0.09,0.13,0.11,0.09,0.02,0.15,0.05,0.14,0.11
3,NaN,2008,0.06,0.14,0.12,0.07,0.15,0.1,0.11,0.08,0.14,0.05,0.13,0.18,0.11
4,NaN,2009,0.09,0.03,0.05,0.13,0.16,0.16,0.11,0.33,0.12,0.18,0.07,0.29,0.14
5,NaN,2010,0.07,0.21,0.24,0.09,0.08,0.07,0.2,0.07,0.12,0.16,0.06,0.1,0.12


In [ ]:
precip2=precip[['Year', 'Jan']]

KeyError: "None of [Index(['Year', 'Jan'], dtype='object')] are in the [columns]"